# Train a 30B LLaMa model
We first need to install some pull requests, rather than the regular transformers libraries. In general this is a terrible idea, but I suppose we are on the bleeding edge here, so we have to take some risks.  The two PRs are from Jason Phang.

In [1]:
!pip install git+https://github.com/huggingface/peft.git@refs/pull/160/head --quiet
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/21955/head --quiet
!pip install sentencepiece --quiet
!pip install numexpr==2.7.3 --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
!cat /home/ubuntu/five-dollar-test/how.json /home/ubuntu/five-dollar-test/what.json > /home/ubuntu/five-dollar-test/train.json 
import transformers 
import json
import numpy as np
import random
import tqdm.auto as tqdm
import datasets

def proc():
    save_path = "/home/ubuntu/llama_save-30"
    tokenizer_path = "decapoda-research/llama-30b-hf"
    data_path = "/home/ubuntu/five-dollar-test/train.json"
    max_seq_length = 448 # 512 does not fit, alas
    
    tokenizer = transformers.LlamaTokenizer.from_pretrained(tokenizer_path)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    all_tokenized = []
    print("reading")
    with open(data_path, "r") as f:
        lines = list(map(json.loads, f.readlines()))
    random.shuffle(lines)
    print("tokenizing")
    for elem in tqdm.tqdm(lines[0:5000]):
        try:
            all_tokenized.append( tokenizer.encode( (elem["input"] + "\n\n" +  elem["output"])[:512],
                                                       #truncation=True,max_length= max_seq_length-2, padding='max_length' )
                                                  ))
            #if  len(all_tokenized[-1]) != max_seq_length -2:
            #    print(len(all_tokenized[-1]), " should be ", max_seq_length-2 )
        except:
            print("bad line", elem)
            break
            
    all_tokens = [tokenizer.bos_token_id] + [
        tok
        for row in all_tokenized
        for tok in row + [tokenizer.eos_token_id, tokenizer.bos_token_id]
    ] + [tokenizer.eos_token_id]

    truncated_tokens = all_tokens[:(len(all_tokens) //  max_seq_length) *  max_seq_length]
    arr = np.array(truncated_tokens).reshape(-1,max_seq_length)
    ds = datasets.Dataset.from_dict({"input_ids": arr})
    #arr = np.array(all_tokenized) 
    #ds = datasets.Dataset.from_dict({"input_ids": arr})
    ds.save_to_disk(save_path)
    print(f"Generated {arr.shape[0]} samples.")
    
proc()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


reading
tokenizing


  0%|          | 0/5000 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/1501 [00:00<?, ? examples/s]

Generated 1501 samples.


In [2]:
import os
import math
#from dataclasses import dataclass, field
import tqdm.auto as tqdm
#from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch.utils.data import Dataset

import datasets
import transformers
from transformers import (
    Trainer,
    TrainingArguments,
)
from peft import (
    get_peft_model,
    LoraConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
    TaskType,
)

def get_peft_config(lora_rank):
       return LoraConfig(
            task_type=TaskType.CAUSAL_LM, 
            inference_mode=False,
            r=lora_rank,
            lora_alpha=32, 
            lora_dropout=0.1,  # we use 10% dropout instead of 5%, because, why not?
       )

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        model_out = model(
            input_ids=inputs["input_ids"],
            attention_mask=torch.ones_like(inputs["input_ids"]),
            labels=inputs["input_ids"],  # HF model does the slicing for us
        )
        if return_outputs:
            return model_out.loss, model_out
        else:
            return model_out.loss

def data_collator(features: list) -> dict:
    return {
        "input_ids": torch.stack([
            torch.LongTensor(f["input_ids"])
            for f in features
        ])
    }


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)

    
def train_llama():
    model_path="decapoda-research/llama-30b-hf"
    dataset_path="/home/ubuntu/llama_save-30/"
    lora_rank = 16
  
    training_args = TrainingArguments(per_device_train_batch_size = 1,
                                        gradient_accumulation_steps = 1,
                                        max_steps = 2000,
                                        learning_rate = 2e-4,
                                        fp16 = True,
                                        logging_steps = 100,
                                        output_dir = "/home/ubuntu/",
                                        optim="adamw_torch",
                                        )

    print("Setup Data")
    dataset = datasets.load_from_disk(dataset_path)

    print("Setup Model")
    model = transformers.LlamaForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 ,
        #load_in_8bit=True,
        device_map="auto",
    )
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)
    model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

    print("Setup PEFT")
    peft_config = get_peft_config(lora_rank)
    model = get_peft_model(model, peft_config)
    model = model.cuda()

    print("Train")
    trainer = ModifiedTrainer(
        model=model,
        train_dataset=dataset,
        args=training_args,
        data_collator=data_collator,
    )
    trainer.train()
    save_tunable_parameters(model, os.path.join(training_args.output_dir, "params-30.p"))


In [ ]:
train_llama()

Setup Data
Setup Model


Loading checkpoint shards:   0%|          | 0/61 [00:00<?, ?it/s]

Setup PEFT
Train


Step,Training Loss


In [1]:
import torch
import transformers
from peft import get_peft_model,LoraConfig
    
def get_peft_config(lora_rank):
       return LoraConfig(
            task_type='lora',
            inference_mode=False,
            r=lora_rank,
            lora_alpha=32, 
            lora_dropout=0.1,
        )
peft_path = "/home/ubuntu/params-30.p"

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = transformers.LlamaForCausalLM.from_pretrained( "decapoda-research/llama-30b-hf")
peft_config = get_peft_config(16)
model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

tokenizer = transformers.LlamaTokenizer.from_pretrained("decapoda-research/llama-30b-hf")







Loading checkpoint shards:   0%|          | 0/61 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [2]:
def f(str):
    inputs = tokenizer(str + "\n\n", return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids=inputs,
                             max_length=500,
                             num_beams=2,
                             repetition_penalty=2.5,
                             length_penalty=0.5,
                             early_stopping=True,
                             no_repeat_ngram_size=2,
                             use_cache=True,
                             do_sample = True,
                             temperature = 1.5,
                             top_k = 50,
                             top_p = 0.95)
    print(tokenizer.decode(outputs[0]))
    


In [3]:
f("Explain why Tom Cruise is so successful")

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 Explain why Tom Cruise is so successful

Do you like this post; can you share with your friends and 1 like to our official Instagram (https://www.instagram.com/celebritystarnews?r=n), I will continue to update the latest TOM CRUISE's gossip, photo scandals and show exclusive videos.
Thank you! CelebrityStarNews [Official Insta] In addition to his excellent acting skills, there are also personal factors that help Tom Cruise keep his place in Hollywood for such a long time. We know some secrets about the pers


In [4]:
f("does the white house have a pool")

 does the white house have a pool

The White Houses indoor swimming pool, which was closed by President Barack Obama and his family after they moved in, is back in business under the Biden administration. The Obamas famously decided to shutter the facilities during their eight years in the executive mansion out of consideration for the security disruptions pools require. But theres nothing stopping new occupants from making use of the 25-yard (23-meter) pool that has been on the property since 1933.


In [5]:
f("explain the reasons for the decline of the Ottoman Empire")

 explain the reasons for the decline of the Ottoman Empire

This assignment is due on Wednesday, March 31, 2021 by 11:59 PM EST (or timely submission as allowed by school policy). Submit your response in Taskstream Dropbox. Your response should be about 750-100 words typed and double-spaced using APA format with a standard cover page. Include citations from at least three sources to support your writing. Provide a References page that corresponds with the in-text citation. Use Times New Roman font size 


In [6]:
f("can a prolactinoma be treated with drugs")


 can a prolactinoma be treated with drugs

Brain tumors are abnormal growths of tissue in the brain.  There are several types of brain tumor, depending on where they grow and the type of cells involved. This information is about pituitary adenomas.
*: Pituicytomas - very rare, slow-growing tumours that do not produce hormones (endocrine) or cause any other symptoms besides visual loss. They are usually non-cancerous (benign). A typical treatment is surgery to remove the tumour. It is uncommon for this type


In [7]:
f("how do you make cheese")

 how do you make cheese

This is a great question  and one that we get asked all the time.
*: Yes, its absolutely possible to make your own cheesecake without dairy products, but there are a few factors to keep in mind. See our list of creamy, plant-based alternatives below! (Image credit: Shannon Aronson via Serious Eats ) Homemade cheescakes can be some of the most rewarding desserts to whip up from scratch. But they can also quickly become frustrating if things dont go exactly as planned. If you


In [8]:
f("list of disney animated movies")


 list of disney animated movies

*: In 1986, The Great Mouse Detective, based on the children's book series Basil of Baker Street by Eve Titus, became Disney' s first animated feature to utilize computer-generated imagery ( CGI ). [12] [edit] Renaissance era (1


In [9]:
f("classification of hormones")


 classification of hormones

Human endocrine system, hypothalamus, pituitary gland, thyroid glands, pancreas, ovaries, and testes. (National Cancer Institute) Wikipedia The human body produces several types of naturally-occurring chemicals called h


In [10]:
f("genres of jazz music")

 genres of jazz music

Top 10 most popular styles of Jazz Music explained and compared to each other: Dixieland, Ragtime, Swing, Bebop, Cool Jazz, Hard Bop, Post-bop & Funky Hardbone, On-Base Avant-Garde, Smooth Jazz. Evolutionary Tree Diagram. In this article we will tell you about the main varieties of modern jazz, give their brief characteristics and compare them with each othe


In [11]:
f("explain the weaknesses of the articles of confederation")

 explain the weaknesses of the articles of confederation

Articles of Confederation, accord signed by 56 delegates to the Continental Congress on October 31, 1778, that served as the United States of Americas first constitution.
*: The U.S. Constitution - Article Side-by-side comparison (English) of original and current versions of Article 4 Section 2 from The University of Chicago


In [12]:
f("explain how to overcome barriers to communication")

 explain how to overcome barriers to communication

*: What are the main types of verbal and non-verbal communication? (See Table 1 in the ESI section) Barriers that can hinder your ability to communicate effectively. (Please see Section 2) How to improve your communication skills by overcoming some of these barrier


In [13]:
f("a story about persephone")

 a story about persephone

A tale of love and devotion, loss and transformation, Persephones abduction at the hands of Hades is one of the most well-known Greek myths in popular culture. But what do we really know about her? Read on to find out more about the Goddess of Spring and why you shouldnt be afraid of her if you see her in your nightmares! The daughter of Zeus and his wife Demeter (Goddes


In [14]:
f("a letter to a sunday keeper")

 a letter to a sunday keeper

To the church members that feel judged for worshiping on Sunday instead of Saturday, this letter is for you: I know what its like to be made to feel as if youre not good enough or spiritual enough because you dont keep the Sabbath on Saturday (Friday sunsets to Saturday sunrises). I remember the guilt and feeling of condemnation when people in my church told me that I wasnt going to heaven if I continued to worship God on Sundays. They said that only those who worsh


In [34]:
f("a letter to bank manager")

 a letter to bank manager

*: How to write an application for a loan from the bank, including what you should include in your letter or email and how it should be formatted. If you're thinking about taking out a mortgage with HSBC, read our expert guide to find out more about its different deals, eligibility criteria and customer service. We explain how to apply for one of these loans, as well as who they might be best suited to. An unsecured personal loan allows you to borrow money without having to put up any collateral (such as your home). A good example of when you might need this kind of loan is if you want to take a holiday but dont have the necessary funds. You can then pay back the loan over a set period - usually several years. One of the benefits of applying for this type of credit is that you wont have to worry about losing your house if something happens and you cant keep up repayments. However, lenders are likely to check your credit history before approving your applicati

In [35]:
f("a letter to the hostel warden for leave")

 a letter to the hostel warden for leave

*: Dear Hostel Warden, I am writing to request permission to take leave due to my unavoidable circumstances. Please grant me leave for 10 days starting from (date of leave) to ( date of return). The reason for my leave is that I have been suffering from stomach pain and fever for the past few days. I went to see a doctor at the nearby clinic who asked me to get admitted in the hospital for further treatment. Therefore, as you can understand, it will not be possible for me attend classes or college activities during this time period. Kindly consider my application and let me know about your decision soon. Thanking You Yours Sincerely, Signature


In [36]:
f("an example of checks and balances")

 an example of checks and balances

The Constitution grants the federal government certain enumerated powers, which are spelled out in Article I, Section 8.
*: The power to collect taxes; these must be uniform (the same) throughout the United States. This means that every state pays the same percentage of its income for a particular tax. Congress can determine what type of tax is collected and how it is spent. Taxes have historically been used to pay off national debt, fund infrastructure projects, provide social welfare programs, and more. Federal tax dollars also fund public education, military expenses, space exploration, transportation, scientific research, housing assistance, environmental protections, foreign aid, disaster relief, Social Security, Medicaid, Medicare, unemployment benefits, veteran rehabilitation services, farm subsidies, immigration regulation, health insurance, prison operations, food safety inspection, drug interdiction, law enforcement, border patrol, water pu

In [37]:
f("an algorithm to find the average of three numbers")

 an algorithm to find the average of three numbers

1: If n is a positive integer, then the arithmetic mean of the first n positive integers is given by the formula where k ranges from 1 to n.
2: The middle term of any arithmetic series that has at least three terms can be determined by averaging the two terms that come before and after it.


In [38]:
f("a* algorithm tic tac toe")

 a* algorithm tic tac toe

Today we look at how to use the minimax algorithm to create an unbeatable AI for Tic-Tac-Toe that will never lose a game. This article follows on from our previous articles where we looked at some basic concepts of Artificial Intelligence and then went on to implement a few search algorithms like Breadth First Search, Depth first search and Dijkstra's Algorithm. Today we build upon these concepts and dive into the Minimax Algorithm which is used in games like Chess and Go but can be applied to any 2 player perfect information strategy game such as checkers or even tictactoe. To do this we simply run a loop through all possible permutations of moves until we find one that does not have a move that allows the opponent to make a winning move. If we are unable to find a valid move then it means that there is no way to win so we return None. The problem with greedy algorithms is that they don't take into account all available options when making a decision. In oth

In [39]:
f("the difference between civil rights and civil liberties")

 the difference between civil rights and civil liberties

Civil rights are defined by the U.S. Department of Labor's Employment and Training Administration (ETA) as "all individuals who work for salaries, wages, or other remuneration." Civil rights laws apply to all job applicants and employees on the basis of race, color, religion, sex (including pregnancy), national origin, age, disability, genetic information, citizenship status, and military service.
1: The Fair Labor Standards Act (FLSA) requires employers to comply with minimum wage, overtime pay, child labor, recordkeeping requirements, etc. 3: Title VII of the Civil Rights Act of 1964 prohibits discrimination in hiring, promotion, termination, harassment, compensation, training, fringe benefits, layoffs, retaliation/retaliatory discharge, working conditions, failure to accommodate, a hostile work environment, denial of opportunities, demotion, discipline, unfair scrutiny, adverse action, differential treatment, glass ceilings, 

In [40]:
f("the difference between zucchini and cucumber")

 the difference between zucchini and cucumber

If you've ever taken a stroll through your local grocery store or farmers market, you may have noticed some confusion surrounding these two similar-looking vegetables. While there are slight differences between them, they are often considered interchangeable substitutes in many recipes. However, it's worth knowing exactly what makes each of these unique to help you decide which is right for your next dish. Zucchinis ( Cucurbita pepo ) are dark green squashes that can reach up to 30 inches in length. They belong to the same family as pumpkins, watermelons, gourds, and other types of summer crops. The flesh of a zuchinni has a mild flavor with an almost meaty consistency. This versatile ingredient can be baked, steamed, boiled, grilled, sauteed, or roasted into a variety of delicious side dishes like this Lemon Zoodles recipe. Aside from their culinary uses, raw sliced zukes also make excellent keto diet snacks due to their low carb content.

In [41]:
f("script for a wedding reception")

 script for a wedding reception

Wedding Scripts Sample Wedding Ceremony Script It is an honor and joy to celebrate with you as your family and friends witness the covenant you are making today. As you enter married life, you bring unique personalities, backgrounds, experiences, hopes, and dreams into this relationship. You have much to be thankful for and so many gifts to share with one another. Marriage can sometimes be difficult, but it has the potential to continually draw you closer to each other while strengthening your relationship with God. With all of these factors in mind, I invite you to join hands and face one-another. Please repeat after me: In the sight of God and these witnesses, we ________ and __________ promise to love, honor, cherish, respect, encourage, support, comfort, pray for, trust, forgive, serve, listen to, protect, nurture, sacrifice for… (and add any additional promises you would like to make)…this day and forevermore.


In [42]:
f("what kind of car is lightning mcqueen")

 what kind of car is lightning mcqueen

Car enthusiasts can debate for days on end about which cars are the most iconic in cinematic history. But we'd argue that there's one clear choice at the top of the list: Lightning McQueen from Cars (2006), Piston Cup winner and former Radiator Springs resident. So, as fans gear up for "Cars 3" this weekend, we thought it might be fun to answer the question that has bedeviled them since they first saw the film: What type of race car does he represent?
*: Dale Earnhardt Jr. - No. 8 Chevrolet Monte Carlo SS The legendary driver made his debut in NASCAR'S Busch Series in 1996, piloting a green-and-white Chevy before making the jump to full-time competition in the Cup Series two years later. He was known for both his talent behind the wheel and his popularity among fans, who affectionately referred to him as "Junior. " Although he retired from racing in November after more than two decades with Hendrick Motorsports, he continues to play an active rol

In [43]:
f("what was the great compromise")

 what was the great compromise

*: The Missouri Compromise (1820): Admitted Maine and Missouri into the Union as a free state and a slave state, respectively; extended the 36°30 line westward through the rest of the Louisiana Purchase to the Pacific Ocean. *: Nullification Crisis : John Calhoun supported South Carolina's attempt to nullify the tariffs of 1928 and '32 on the grounds that such taxes were unconstitutional. Although President Andrew Jackson threatened military action if any state attempted to secede from the union, this issue was resolved peacefully by Henry Clay with the Tariff of Abominations in , which lowered the rate of import duties.
o: This crisis showed how fragile the balance of power between North and South really was. o: It also illustrated the need for a more permanent solution to slavery than the Missouri agreement. There were many differences between the north and the south before the civil war but one of them was whether or not new states coming into USA sho

In [49]:
f("what can mimic kidney stone pain")

 what can mimic kidney stone pain

Kidney stones are hard masses of salts and minerals that form in the kidneys when there is a decrease in urine volume and/or an excess of stone-forming substances. There are different types of kidnesy stones. The most common type is made of calcium oxalate. Other types include uric acid, struvite (magnesium, ammonium and phosphate), cystine and silicate.
*: Severe back, side (flank) or abdominal pain; pain radiating to the groin and genitals may also occur.  Pain often starts suddenly and comes in waves lasting 20 to 60 minutes at a time. Sometime pain subsides on its own only to recur days later. Symptoms usually occur when a stone drops into the ureter, causing irritation or blockage. If you have any of these symptoms, make an appointment to see your health care provider. Your provider will review your medical history and ask you about your diet, lifestyle and past health problems. You will be asked if you use medications for blood pressure, cholest

In [45]:
f("how is jello made")

 how is jello made

Jell-O  is a brand name that was created in 1897 by carpenter Pearle Bixby Wait to sell his patented gelatin desserts. It's not the same thing as gelatine or agar, although they are very similar and can be used interchangeably in recipes. Gelatin comes from collagen, one of the main proteins found in connective tissue such as skin, bones, hooves, and hides. The word "gelatin" itself is derived from the Latin word for animal glue. Agar is also a common culinary thickening agent; it comes exclusively from seaweed (usually red algae), rather than an animal source. In addition to being available in powdered form, these ingredients are also sold in sheet form. Both types of sheets are sometimes referred to as "agar strips" or "gelfilm. " They're useful for adding garnish to cakes and other dishes because they don't dissolve until after you cook them. To make your own versions of gelled foods, you need to know a little chemistry. Jellied food relies on hydrocolloids, whic

In [46]:
f("how is osteoporosis diagnosed")

 how is osteoporosis diagnosed

A bone density test can determine if your bones are at risk of breaking. Learn more about getting tested for this silent disease. A DEXA scan is the most common test to measure a persons bonal density, but other tests can be used as well. Find out who should get tested and what options are available. Often called the silent thief, osteoarthritis causes cartilage between joints to break down.
*: What causes it?
1: Who's affected? 2: Symptoms 3: Treatment 4: Prevention


In [47]:
f("how are viruses different from bacteria")

 how are viruses different from bacteria

Bacterial and viral infections have similar symptoms like fever, sore throat, cough, diarrhea, vomiting, fatigue, and aches. However, these two types of infection differ in the following aspects:
1: Duration of Infection Bacterias take longer to replicate than a virus. You can get rid of a bacteraemia through antibiotics in 3-5 days. It takes several weeks to completely recover from an influenza or common cold (caused by rhinoviruses) because they need time to kill the virus inside your body system.
2: Mode of Transmission There are three ways you can be infected with a disease-causing agent: airborne, food/water, or direct contact. Viruses spread mainly through the air when people sneeze or couch around you. They can also spread if you touch contaminated surfaces and then touch your mouth, nose, eyes, etc., without first washing your hands. This mode of transmission is called fecal-oral route. On the other hand, some diseases caused by baceter

In [50]:
f("how is the moon illuminated")

 how is the moon illuminated

The Moon is not a light source, and it does not generate its own light.
*: The Sun lights up the surface of the Moon (the "dayside") so that we can see it in our nighttime sky. *: A Full Moon occurs when the full face of this lit-up side is pointed directly at us. As the Earth revolves around the Sun each month, the same hemisphere faces away from the star. When the half facing towards the sun is visible again, it is called a Waxing Crescent Moon. If you look closely at the image above, you'll notice that the shadows on the Moons surface aren't black. This is because the entire Universe is essentially filled with electromagnetic radiation, most of which is invisible to the human eye. We cannot see infrared or ultraviolet wavelengths, for example, but they do pass through the atmosphere into outer space where they are reflected off the Mooon's surface. Even though the lunar body is about 384,00 kilometers away, some of these photons still make it back to Ea